# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet
#tfdet.core.util.set_seed(777) #set seed
device = tfdet.core.util.select_device(0) #set device

if True: #apply mixed precision
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

# 1. Init Dataset

In [3]:
import os
path = "./balloon"
if not os.path.exists(path):
    !wget -qq --no-check-certificate http://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
    !unzip -qq balloon_dataset.zip
print(tfdet.dataset.util.tree_dir(path))

balloon/
    train/
        605521662_a470fef77f_b.jpg
        9330497995_4cf0438cb6_k.jpg
        5178670692_63a4365c9c_b.jpg
        ...
    val/
        4838031651_3e7b5ea5c7_b.jpg
        16335852991_f55de7958d_k.jpg
        24631331976_defa3bb61f_k.jpg
        ...


In [4]:
tr_path = os.path.join(path, "train/via_region_data.json")
te_path = os.path.join(path, "val/via_region_data.json")

label = tfdet.dataset.balloon.LABEL
color = None

n_class = len(label)
image_shape = [448, 448]

tr_dataset = tfdet.dataset.balloon.load_dataset(tr_path, mask = False,
                                                cache = None)
tr_dataset = tfdet.dataset.Dataset(tr_dataset,
                                   transform = [{"name":"load"},
                                                {"name":"weak_augmentation", "crop_shape":None},
                                                {"name":"resize", "image_shape":image_shape},
                                                {"name":"pad", "image_shape":image_shape},
                                                {"name":"filter_annotation"},
                                                {"name":"label_encode", "label":label},
                                                {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]}])

te_dataset = tfdet.dataset.balloon.load_dataset(te_path, mask = False,
                                                cache = None,
                                                transform = [{"name":"load"},
                                                             {"name":"resize", "image_shape":image_shape},
                                                             {"name":"pad", "image_shape":image_shape},
                                                             {"name":"filter_annotation"},
                                                             {"name":"label_encode", "label":label},
                                                             {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]}])

tr_pipe = tfdet.dataset.PipeLoader(tr_dataset)
tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe, keys = ["x_true", "y_true", "bbox_true", "mask_true"])

te_pipe = tfdet.dataset.PipeLoader(te_dataset)
te_pipe = tfdet.dataset.pipeline.args2dict(te_pipe, keys = ["x_true", "y_true", "bbox_true", "mask_true"])

Preprocessing Data: 100%|██████████| 13/13 [00:00<00:00, 13.28it/s]


# 2. Build Detector

In [5]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    out = tfdet.model.detector.effdet_lite_d2(x, n_class = n_class,
                                              scale = [24, 48, 96, 192, 336], ratio = [0.5, 1, 2], octave = 3,
                                              weights = "imagenet")
    
    model = tfdet.model.train.effdet.train_model(x, *out, assign = tfdet.core.assign.max_iou,
                                                 proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.25)

# 3. Train

In [6]:
import shutil

epoch = 300
batch_size = 16
update_step = 64 // batch_size
save_path = "./learn/epoch@{epoch:03d}-metric@{mean_average_precision:.4f}-loss@{loss:.4f}.h5"

learning_rate = 1e-2
decay_rate = 1e-2 #dst learning_rate = learning_rate * decay_rate
momentum = 0.9
nesterov = True
warm_up_epoch = 5

if os.path.exists(os.path.dirname(save_path)):
    shutil.rmtree(os.path.dirname(save_path))
os.makedirs(os.path.dirname(save_path), exist_ok = True)

train_pipe = tfdet.dataset.pipeline.key_map(tr_pipe, map = {"x_true":x.name}, repeat = 8, batch_size = batch_size, shuffle = True, prefetch = True) #61 * 8 > 488
test_pipe = tfdet.dataset.pipeline.key_map(te_pipe, map = {"x_true":x.name}, batch_size = batch_size, prefetch = True)

total_step = len(train_pipe) #int(train_pipe.cardinality())
total_test_step = len(test_pipe) #int(test_pipe.cardinality())

ema = tfdet.callback.EMA(step = update_step, auto_apply = True, warm_up_epoch = warm_up_epoch)
metric = tfdet.callback.CoCoMeanAveragePrecision(test_pipe, label = label, name = "mean_average_precision")
save = tf.keras.callbacks.ModelCheckpoint(save_path, monitor = "mean_average_precision", mode = "max", save_best_only = True, save_weights_only = True, save_freq = "epoch", verbose = 0)
#scheduler = tfdet.callback.WarmUpCosineLearningRateScheduler(cycle = epoch, decay_rate = decay_rate, warm_up_epoch = warm_up_epoch)
scheduler = tfdet.callback.WarmUpCosineLearningRateSchedulerStep(cycle = epoch, step = update_step, decay_rate = decay_rate, total_step = total_step, warm_up_epoch = warm_up_epoch) #total step None > dynamic total step
logger = tf.keras.callbacks.CSVLogger(os.path.join(os.path.dirname(save_path), "logger.csv"), separator = ",")

In [7]:
with device:
    #optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov, clipnorm = 1.)
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
    if tf.keras.mixed_precision.global_policy().compute_dtype == "float16":
        optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
    model.compile(optimizer = optimizer)

    model.fit(train_pipe,
              #validation_data = test_pipe,
              epochs = epoch, steps_per_epoch = total_step, validation_steps = total_test_step,
              callbacks = [ema, metric, save, scheduler, logger],
              verbose = 2)

Epoch 1/50
32/32 - 125s - loss: 284.9377 - loss_class: 283.0276 - loss_bbox: 1.9097 - ema_n_update: 32.0000 - mean_average_precision: 0.0000e+00 - mean_average_precision@.5: 0.0000e+00 - mean_average_precision@.75: 0.0000e+00 - learning_rate: 0.0020 - 125s/epoch - 4s/step
Epoch 2/50

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |49      |0.058 |0.0145           |0.0539              |0.0004               |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |49      |0.058 |0.0145           |0.0539              |0.0004               |
+----------+--------+-

# 4. Evaluate

In [8]:
model_path = tfdet.dataset.util.list_dir(os.path.dirname(save_path), "h5")
model_path = sorted(model_path, key = lambda x: x.split("metric@")[1].split("-")[0], reverse = True)[0]

with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    out = tfdet.model.detector.effdet_lite_d2(x, n_class = n_class, scale = [24, 48, 96, 192, 336], ratio = [0.5, 1, 2], octave = 3)
    nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.25)(out)
    model = tf.keras.Model(x, nms_out)
    model.load_weights(model_path)
    
metric = tfdet.core.metric.CoCoMeanAveragePrecision(label = label)
for te_dict in test_pipe:
    x_true, y_true, bbox_true = te_dict["x_true"], te_dict["y_true"], te_dict["bbox_true"]
    y_pred, bbox_pred = model.predict(x_true, verbose = 0)[:2]
    metric.add(y_true, bbox_true, y_pred, bbox_pred)
print(metric.summary_text)

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |40      |0.394 |0.3736           |0.5337              |0.4969               |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |40      |0.394 |0.3736           |0.5337              |0.4969               |
+----------+--------+--------+------+-----------------+--------------------+---------------------+


# 5. Visualize

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

out = te_dataset[0] #or next(iter(tr_dataset))
x_true, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

out = model.predict(np.expand_dims(x_true, axis = 0), verbose = 0)
y_pred, bbox_pred = [o[0] for o in out[:2]]
mask_pred = out[2][0] if 2 < len(out) else None

x_true = tfdet.dataset.transform.unnormalize(x_true, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375])
x_pred = tfdet.util.draw_bbox(x_true, bbox_pred, y_true = y_pred, mask_true = mask_pred, label = label, color = color, crop_mask = False)
x_true = tfdet.util.draw_bbox(x_true, bbox_true, y_true = y_true, mask_true = mask_true, label = label, color = color)

fig_size = (10, 10)
plt.figure(figsize = fig_size)
plt.imshow(x_true)
plt.figure(figsize = fig_size)
plt.imshow(x_pred)